In [1]:
spark

In [5]:
header = sc.textFile('kktv-17-11/train').take(1)[0]

In [11]:
header

u'user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer'

In [7]:
train_evts = sc.textFile('kktv-17-11/train').filter(lambda e: e != header)

In [8]:
train_evts.count()

57661193

In [10]:
from pyspark.sql import Row

In [12]:
import datetime as dt

In [14]:
len('2017-06-08 14:54:30.325')

23

In [15]:
'2017-06-08 14:54:30.325'[:19]

'2017-06-08 14:54:30'

In [23]:
test_dt = dt.datetime.strptime('2017-06-08 14:54:30.325'[:19], '%Y-%m-%d %H:%M:%S')

In [29]:
dt.datetime(2017, 1, 1, 9, 13, 0) < dt.datetime(2017, 1, 1, 9, 13, 45)

True

In [27]:
test_dt.month, test_dt.day, test_dt.hour

(6, 8, 14)

In [73]:
test_dt.

3

In [40]:
header.split(',')

[u'user_id',
 u'device_id',
 u'session_id',
 u'title_id',
 u'event_time',
 u'played_duration',
 u'action_trigger',
 u'platform',
 u'episode_number',
 u'series_total_episodes_count',
 u'internet_connection_type',
 u'is_trailer']

In [43]:
len(header.split(','))

12

In [38]:
def parse_slot(d):
    slot = 3
    if d < dt.datetime(d.year, d.month, d.day, 9, 0, 0):
        slot = 0
    elif d < dt.datetime(d.year, d.month, d.day, 17, 0, 0):
        slot = 1
    elif d < dt.datetime(d.year, d.month, d.day, 21, 0, 0):
        slot = 2
    return slot

In [64]:
def parse_evt(p):
    evt_time = dt.datetime.strptime(p[4][:19], '%Y-%m-%d %H:%M:%S')

    r = Row(
        user_id=p[0],
        device_id=p[1],
        session_id=p[2],
        title_id=p[3],
        event_time=evt_time,
        played_duration=float(p[5]),
        action_trigger=p[6],
        platform=p[7],
        episode_number=int(p[8]),
        series_total_episodes_count=int(p[9]),
        internet_connection_type=p[10],
        is_trailer=bool(p[11]),
        month=evt_time.month,
        day=evt_time.day,
        hour=evt_time.hour,
        slot=parse_slot(evt_time)
    )
    return r

In [66]:
train_df = train_evts.map(lambda e: e.split(',')).map(parse_evt).toDF()

In [67]:
train_df.printSchema()

root
 |-- action_trigger: string (nullable = true)
 |-- day: long (nullable = true)
 |-- device_id: string (nullable = true)
 |-- episode_number: long (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- hour: long (nullable = true)
 |-- internet_connection_type: string (nullable = true)
 |-- is_trailer: boolean (nullable = true)
 |-- month: long (nullable = true)
 |-- platform: string (nullable = true)
 |-- played_duration: double (nullable = true)
 |-- series_total_episodes_count: long (nullable = true)
 |-- session_id: string (nullable = true)
 |-- slot: long (nullable = true)
 |-- title_id: string (nullable = true)
 |-- user_id: string (nullable = true)



In [68]:
train_df.select('user_id', 'event_time', 'month', 'day', 'hour', 'slot').sample(False, 0.1).show(100)

+-------+-------------------+-----+---+----+----+
|user_id|         event_time|month|day|hour|slot|
+-------+-------------------+-----+---+----+----+
|      0|2017-06-08 14:54:34|    6|  8|  14|   1|
|      0|2017-06-08 15:18:25|    6|  8|  15|   1|
|      0|2017-06-08 15:21:35|    6|  8|  15|   1|
|      0|2017-06-08 15:28:44|    6|  8|  15|   1|
|      0|2017-06-10 12:34:37|    6| 10|  12|   1|
|      0|2017-06-10 12:34:57|    6| 10|  12|   1|
|      0|2017-06-11 07:10:26|    6| 11|   7|   0|
|      0|2017-06-11 07:12:01|    6| 11|   7|   0|
|      0|2017-08-19 01:40:44|    8| 19|   1|   0|
|      2|2017-07-30 03:05:03|    7| 30|   3|   0|
|      2|2017-08-17 09:23:43|    8| 17|   9|   1|
|      3|2017-05-28 15:40:14|    5| 28|  15|   1|
|      3|2017-05-28 15:41:33|    5| 28|  15|   1|
|      3|2017-05-28 15:55:41|    5| 28|  15|   1|
|      3|2017-05-28 16:07:15|    5| 28|  16|   1|
|      3|2017-05-28 16:32:35|    5| 28|  16|   1|
|      3|2017-05-28 16:43:49|    5| 28|  16|   1|


In [71]:
train_df.where('hour > 21').select('user_id', 'event_time', 'month', 'day', 'hour', 'slot').show()

+-------+-------------------+-----+---+----+----+
|user_id|         event_time|month|day|hour|slot|
+-------+-------------------+-----+---+----+----+
|      4|2017-07-06 22:48:28|    7|  6|  22|   3|
|      4|2017-07-06 22:51:22|    7|  6|  22|   3|
|      4|2017-07-06 22:51:32|    7|  6|  22|   3|
|      4|2017-07-06 22:51:41|    7|  6|  22|   3|
|      4|2017-07-06 22:59:03|    7|  6|  22|   3|
|      4|2017-07-06 23:02:32|    7|  6|  23|   3|
|      4|2017-07-06 23:07:34|    7|  6|  23|   3|
|      4|2017-07-06 23:33:33|    7|  6|  23|   3|
|      4|2017-07-06 23:40:04|    7|  6|  23|   3|
|      4|2017-07-06 23:42:34|    7|  6|  23|   3|
|      4|2017-07-06 23:48:43|    7|  6|  23|   3|
|      4|2017-07-06 23:56:21|    7|  6|  23|   3|
|      4|2017-07-09 23:59:59|    7|  9|  23|   3|
|      4|2017-07-10 23:41:48|    7| 10|  23|   3|
|      4|2017-07-10 23:41:52|    7| 10|  23|   3|
|      4|2017-07-10 23:41:55|    7| 10|  23|   3|
|      4|2017-07-10 23:41:59|    7| 10|  23|   3|


In [ ]:
user_evts = train_df.rdd.groupBy(lambda e: e.user_id).take(1)[0]

In [ ]:
# 2017/1/2 - 2017/8/13